In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.pyplot as plt

import plotly.graph_objects as go 
import plotly.express as px 
import seaborn as sns
import scipy.stats as stats
from math import sqrt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<center style="font-family:verdana;"><h1 style="font-size:200%; padding: 10px; background: black;"><b style="color:#03e8fc;">Positive-Pressure Ventilation</b></h1></center>

"Positive-pressure ventilation means that airway pressure is applied at the patient's airway through an endotracheal or tracheostomy tube. The positive nature of the pressure causes the gas to flow into the lungs until the ventilator breath is terminated. As the airway pressure drops to zero, elastic recoil of the chest accomplishes passive exhalation by pushing the tidal volume out."

"It was observed a decrease in mortality rate from 80% to 25%, led to the adaptation of the positive-pressure machines used in the operating room for use in the ICU."

https://emedicine.medscape.com/article/304068-overview

![](https://media.springernature.com/lw685/springer-static/image/art%3A10.1186%2F1749-8090-5-39/MediaObjects/13019_2010_Article_272_Fig2_HTML.jpg)cardiothoracicsurgery.biomedcentral.com

https://cardiothoracicsurgery.biomedcentral.com/articles/10.1186/1749-8090-5-39


"Mechanical ventilation and the total artificial heart: optimal ventilator trigger to avoid post-operative autocycling - a case series and literature review"

Authors: Allen B Shoham, Bhavesh Patel, Francisco A Arabia & Michael J Murray 
Journal of Cardiothoracic Surgery volume 5, Article number: 39 (2010)


"Many patients with end-stage cardiomyopathy are now being implanted with Total Artificial Hearts (TAHs). The authors have observed individual cases of post-operative mechanical ventilator autocycling with a flow trigger, and subsequent loss of autocycling after switching to a pressure trigger. Their data suggest that mechanical ventilator autocycling may be resolved or prevented by the use of a pressure trigger rather than a flow trigger setting in patients with TAHs who require mechanical ventilation."

"In summary, autocycling of the mechanical ventilator occurred in 50% of patients who had received TAHs (Total Artificial Hearts) with the use of a flow trigger ventilator setting." 

"Mechanical ventilator PEEP (Positive end-expiratory Pressure) maintenance maintains pressure at the expense of altered flow, ultimately leading to autocycling in the case of a flow trigger. Given the frequency of autocycling in the ICU, this information may be applicable to other patients who are mechanically ventilated."

"Because ADVANCED VENTILATOR SOFTWARE has significantly diminished differences in inspiratory work of breathing, physicians may consider using a pressure trigger as an initial ventilator mode, or switching to this mode in patients suspected of, or at high risk for autocycling."

https://cardiothoracicsurgery.biomedcentral.com/articles/10.1186/1749-8090-5-39

In [ ]:
train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
test  = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Stages of Mechanical Ventilation</span></h1><br>

"There are four stages of mechanical ventilation. There is the trigger phase, the inspiratory phase, the cycling phase, and the expiratory phase. The trigger phase is the initiation of an inhalation which is triggered by an effort from the patient or by set parameters by the mechanical ventilator."

https://www.ncbi.nlm.nih.gov/books/NBK539742/

In [ ]:
train.head()

#I had to install Bokeh and Holoviews separated so that I could work with those packages. Now, I need some ventilation, after finding out that a code that I ran less than one month ago didn't work.

In [ ]:
!pip install bokeh

In [ ]:
!pip install holoviews

In [ ]:
import holoviews as hv
from holoviews import opts, dim
hv.extension('bokeh')

"Respiratory system compliance and resistance in the critically ill" 

Chapter:Respiratory system COMPLIANCE and RESISTANCE in the critically ill.

Authors: Ricardo Luiz Cordioli and Laurent Brochard - DOI:10.1093/med/9780199600830.003.0074

◆ "Correct understanding of the relationship between pressure, volume, and flow is a basic requirement for correctly setting a ventilator."

◆ "The lungs and chest wall both participate to global respiratory mechanics. In some situations, such as obesity or ARDS, the chest wall could explain a large part of the low compliance of the respiratory system."

◆ "RESISTANCE (R) represents the ratio between the pressure dissipated by the friction of gas and the mean gas flow."

◆ "COMPLIANCE (C) denotes the capacity of the pulmonary system to expand and is calculated as ratio between volume and pressure."

◆ "Intrinsic positive end expiratory pressure (PEEP) can be present with short expiratory time, high respiratory rate, or expiratory flow limitation in airway obstructive disease."

https://oxfordmedicine.com/view/10.1093/med/9780199600830.001.0001/med-9780199600830-chapter-74

In [ ]:
#Code by Kohei-mu https://www.kaggle.com/koheimuramatsu/industrial-accident-causal-analysis/notebook

resist_cnt = np.round(train['R'].value_counts(normalize=True) * 100)
hv.Bars(resist_cnt).opts(title="Respiratory System Resistance", color="green", xlabel="R", ylabel="Percentage", yformatter='%d%%')\
                .opts(opts.Bars(width=500, height=300,tools=['hover'],show_grid=True))

In [ ]:
#Code by Kohei-mu https://www.kaggle.com/koheimuramatsu/industrial-accident-causal-analysis/notebook

compli_cnt = np.round(train['C'].value_counts(normalize=True) * 100)
hv.Bars(compli_cnt).opts(title="Respiratory System Compliance", color="yellow", xlabel="C", ylabel="Percentage", yformatter='%d%%')\
                .opts(opts.Bars(width=500, height=300,tools=['hover'],show_grid=True))

In [ ]:
 #Code By Kohei-mu  https://www.kaggle.com/koheimuramatsu/industrial-machine-anomaly-detection

count = hv.Bars(train.groupby(['C','R'])['u_out'].count()).opts(ylabel="Count", title='Compliance/Resistance Count')
mean = hv.Bars(train.groupby(['C','R']).agg({'u_out': ['mean']})['u_out']).opts(ylabel="Count", title='Compliance/Resistance Mean')
(count + mean).opts(opts.Bars(width=380, height=300,tools=['hover'],show_grid=True, stacked=True, legend_position='bottom'))

#The second control input is a binary variable representing whether the exploratory valve is open (1) or closed (0) to let air out.

In [ ]:
#Code by Kohei-mu https://www.kaggle.com/koheimuramatsu/industrial-accident-causal-analysis/notebook

out_cnt = np.round(train['u_out'].value_counts(normalize=True) * 100)
hv.Bars(out_cnt).opts(title="Air Out", color="blue", xlabel="U_out", ylabel="Percentage", yformatter='%d%%')\
                .opts(opts.Bars(width=500, height=300,tools=['hover'],show_grid=True))

In [ ]:
 #Code By Kohei-mu  https://www.kaggle.com/koheimuramatsu/industrial-machine-anomaly-detection

count = hv.Bars(train.groupby(['R','u_out'])['pressure'].count()).opts(ylabel="Count", title='Resistance/Air out Count')
mean = hv.Bars(train.groupby(['R','u_out']).agg({'pressure': ['mean']})['pressure']).opts(ylabel="Count", title='Resistance/Air out Mean')
(count + mean).opts(opts.Bars(width=380, height=300,tools=['hover'],show_grid=True, stacked=True, legend_position='bottom'))

In [ ]:
#Code By Kohei-mu  https://www.kaggle.com/koheimuramatsu/industrial-machine-anomaly-detection

resist_maxmin = train.groupby(['R','pressure']).agg({'u_out': ['min', 'max']})
(hv.Bars(resist_maxmin['u_out']['max']).opts(ylabel="Count", title='Resistance/Pressure Max') \
+ hv.Bars(resist_maxmin['u_out']['min']).opts(ylabel="Count", title='Resistance/Pressure Min'))\
    .opts(opts.Bars(width=380, height=300,tools=['hover'],show_grid=True, stacked=True, legend_position='bottom'))

In [ ]:
#Code By Kohei-mu  https://www.kaggle.com/koheimuramatsu/industrial-machine-anomaly-detection

compli_maxmin = train.groupby(['C','u_out']).agg({'pressure': ['min', 'max']})
(hv.Bars(compli_maxmin['pressure']['max']).opts(ylabel="Count", title='Compliance/Pressure Max') \
+ hv.Bars(compli_maxmin['pressure']['min']).opts(ylabel="Count", title='Compliance/Pressure Min'))\
    .opts(opts.Bars(width=380, height=300,tools=['hover'],show_grid=True, stacked=True, legend_position='bottom'))

In [ ]:
#Code By Kohei-mu  https://www.kaggle.com/koheimuramatsu/industrial-machine-anomaly-detection
#Density is the distribution density. Not a feature. Hence, don't change it.

hv.Distribution(train['R']).opts(opts.Distribution(title="Resistance Distribution", xlabel="Respiratory System Resistance", ylabel="Density", width=700, height=300,tools=['hover'],show_grid=True))

In [ ]:
#Code By Kohei-mu  https://www.kaggle.com/koheimuramatsu/industrial-machine-anomaly-detection
#Density is the distribution density. Not a feature. Hence, don't change it.

hv.Distribution(train['C']).opts(opts.Distribution(title="Compliance Distribution", xlabel="Respiratory System Compliance", ylabel="Density", color="red", width=700, height=300,tools=['hover'],show_grid=True))

In [ ]:
#Code By Kohei-mu  https://www.kaggle.com/koheimuramatsu/industrial-machine-anomaly-detection

((hv.Distribution(train.loc[train['R']==50,'u_out'], label='5') * hv.Distribution(train.loc[train['R']==5,'u_out'], label='5')).opts(title="Respiratory System Resistance", legend_position='bottom') + \
(hv.Distribution(train.loc[train['R']==50,'u_out'], label='20') * hv.Distribution(train.loc[train['C']==20,'u_out'], label='20') \
     * hv.Distribution(train.loc[train['C']==10,'u_out'], label='10')).opts(title="Respiratory System Compliance", legend_position='bottom')) \
     .opts(opts.Distribution(xlabel="Respiratory System Resistance and Compliance ", ylabel="Density", width=380, height=300,tools=['hover'],show_grid=True))

In [ ]:
#Code By Kohei-Mu  https://www.kaggle.com/koheimuramatsu/women-s-shoes-eda

resi_cnt = train['R'].value_counts() > 100
col = hv.Bars(train['R'].value_counts()[resi_cnt][::-1]).opts(title="Resistance Count over 100", color="purple", xlabel="Respiratory System Resistance", ylabel="Count")\
                .opts(opts.Bars(width=400, height=300,tools=['hover'],show_grid=True,invert_axes=True,fontsize={'title':10}))
pr = hv.Distribution(train['C']).opts(title="Compliance Distribution", color="purple", xlabel="Respiraty System Compliance", ylabel="Density",xformatter='$ %d')\
                            .opts(opts.Distribution(width=400, height=300,tools=['hover'],show_grid=True))
col + pr

The Intraclass Correlation Coefficients (ICC) for the measurements of respiratory system static compliance, interexaminer (A = A1 x B and C = B x A2) and intraexaminer (B = A1 x A2), with n = 66. ICC - intraclass correlation coefficient;  Cst rs - Static Compliance of the Respiratory System

https://www.scielo.br/j/rbti/a/cxtS76H9LgDxd3dR7hXhrDv/?lang=en&format=pdf

In [ ]:
import seaborn as sbn

correlation=train.corr()
plt.figure(figsize=(10,10))
sbn.heatmap(correlation,annot=True,cmap=plt.cm.summer);

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from graphviz import Source


from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split


from scipy.stats import skew
plt.style.use('ggplot')

In [ ]:
#Code by Tejashvi14 https://www.kaggle.com/tejashvi14/casualty-analysis/notebook

#After that snippet the float(Year) and the other objects become integers.

train["time_step"]=train["time_step"].apply(int)
train["u_in"]=train["u_in"].apply(int)
train["pressure"]=train["pressure"].apply(int)

In [ ]:
x = train.drop(['id', 'pressure'], axis=1)
x.fillna(999999, inplace=True)
y = train['pressure']

In [ ]:
dt = DecisionTreeClassifier(max_depth=3)

In [ ]:
dt.fit(x, y)

In [ ]:
dt_feat = pd.DataFrame(dt.feature_importances_, index=x.columns, columns=['feat_importance'])
dt_feat.sort_values('feat_importance').tail(8).plot.barh(figsize=(14, 6), color='green')
plt.show()

In [ ]:
from IPython.display import SVG
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

graph = Source(export_graphviz(dt, out_file=None, feature_names=x.columns, filled = True))
display(SVG(graph.pipe(format='svg')))

![](https://www.ahrq.gov/sites/default/files/wysiwyg/professionals/quality-patient-safety/hais/tools/mvp/modules/vae/ptsoffventilator/ptsoffventilator-slide6.jpg)ahrq.gov

#Acknowledgements: Kohei-Mu  https://www.kaggle.com/koheimuramatsu/women-s-shoes-eda

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Machine Ventilation is still Crude</span></h1><br>

"In just 50 short years we have gone from relatively crude, totally mechanical devices that could provide only machine-triggered volume ventilation to highly evolved microprocessor-controlled systems capable of any form of ventilatory support imaginable." Robert Kacmarek wrote in (2011): “The mechanical ventilator: past, present, and future.”
https://www.rochester.edu/newscenter/brief-history-of-ventilators-424312/

Therefore, let's see how our Kaggle community can improve Mechanical Ventilation with algorithms.